In [1]:
import pandas as pd
import numpy as np

## Functions

In [2]:
def dropper(df):
    nan_value = float("NaN")
    df.replace(0.0, nan_value, inplace=True)
    return df

In [3]:
def formatter(input_df):
    df = input_df
    del df['ID']
    df.columns = ['Origin', 'Destination', 'Year', 'Commuters', 'Distance']
    df['Origin'] = df['Origin'].map(lambda x: x.lstrip('GM'))
    df['Destination'] = df['Destination'].map(lambda x: x.lstrip('GM'))
    df = dropper(df)
    df.dropna(subset=["Commuters"], inplace=True)
    df = df.reset_index()
    del df['index']
    for row in df.iterrows():
        X = row[1]['Year']
        X = X.split('MM')[0]
        Y = row[1]['Commuters']
        Y = Y * 1000
        df.at[row[0], 'Year'] = X
        df.at[row[0], 'Commuters'] = Y
    df['Origin'] = df['Origin'].astype(int)
    df['Destination'] = df['Destination'].astype(int)
    df['Distance'] = df['Distance'].str.split().str.get(-1)
    df['Distance'] = pd.to_numeric(df['Distance'], errors='coerce')
    df['Distance'] = df['Distance'].astype(float)
    return df

In [4]:
def commuting_matrix_creator(input_df, year):
    municipalities = input_df['Origin'].unique().tolist()
    output_df = pd.DataFrame(columns=municipalities, index=municipalities)
    for i in input_df.itertuples():
        output_df.at[i.Origin, i.Destination] = i.Commuters
    output_df = output_df.fillna(0)
    output_df = output_df.sort_index()
    output_df = output_df.sort_index(axis=1)
    pathway = 'Databases/Commuting_Matrix_' + str(year) + '.csv'
    output_df.to_csv(f'{pathway}')
    return output_df

In [5]:
def population_formatter(pathway):
    pop_df = pd.read_csv(f'{pathway}', sep=';')
    del pop_df['ID']
    del pop_df['Sex']
    pop_df.columns = ['Region', 'Year', 'Population']
    pop_df.dropna(subset=["Population"], inplace=True)
    pop_df['Region'] = pop_df['Region'].map(lambda x: x.lstrip('GM'))
    pop_df['Region'] = pop_df['Region'].astype(int)
    pop_df = pop_df.reset_index()
    del pop_df['index']
    for row in pop_df.iterrows():
        X = row[1]['Year']
        X = X.split('JJ')[0]
        pop_df.at[row[0], 'Year'] = int(X)
    return pop_df

In [6]:
def format_for_regression(comm_df, pop_df, year):
    df = pre_formatter(comm_df.copy())
    df2 = pop_df.loc[pop_df['Year'] == int(year)]
    for i in df.itertuples():
        for j in df2.itertuples():
            if i.Origin == j.Region:
                df.at[i.Index, 'Origin Pop'] = np.log(j.Population)
                if i.Destination == j.Region:
                    df.at[i.Index, 'Destination Pop'] = np.log(j.Population)
            elif i.Destination == j.Region:
                df.at[i.Index, 'Destination Pop'] = np.log(j.Population)
    pathway = 'Databases/Regression_DataFrame_' + str(year) + '.csv'
    df.to_csv(f'{pathway}')
    return df

In [7]:
def pre_formatter(input_df):
    df = input_df.copy()
    for i in df.itertuples():
        if i.Distance != 0:
            df.at[i.Index, 'Distance'] = np.log(i.Distance)
        if i.Commuters != 0:
            df.at[i.Index, 'Commuters'] = np.log(i.Commuters)
    return df

In [8]:
def get_info(pathway, year, pop_df):
    rawdf = formatter(pd.read_csv(f'{pathway}', sep=";"))
    df = commuting_matrix_creator(rawdf, year)
    df_reg = format_for_regression(rawdf, pop_df, year)
    return df, df_reg

In [9]:
pop_general = population_formatter('Databases/Population 2014-2020.csv')

In [10]:
com_matrix_14, df_14_reg = get_info('Databases/Mobility 2014.csv', 2014, pop_general)
com_matrix_15, df_15_reg = get_info('Databases/Mobility 2015.csv', 2015, pop_general)
com_matrix_16, df_16_reg = get_info('Databases/Mobility 2016.csv', 2016, pop_general)
com_matrix_17, df_17_reg = get_info('Databases/Mobility 2017.csv', 2017, pop_general)
com_matrix_18, df_18_reg = get_info('Databases/Mobility 2018.csv', 2018, pop_general)
com_matrix_19, df_19_reg = get_info('Databases/Mobility 2019.csv', 2019, pop_general)
com_matrix_20, df_20_reg = get_info('Databases/Mobility 2020.csv', 2020, pop_general)

In [11]:
# aggregate_df_reg = pd.concat([df_14_reg, df_15_reg, df_16_reg, df_17_reg, df_18_reg, df_19_reg, df_20_reg])
# aggregate_df_reg.to_csv('Databases/Test_file.csv', index=False)

## Changes in borders between municipalities

Some years, the Dutch government change municipality's borders by merging small ones into bigger ones for a myriad of reasons. We will identify these year to year modifications in the matrices.

In [12]:
municipalities_14 = com_matrix_14.columns.values.tolist()
municipalities_15 = com_matrix_15.columns.values.tolist()
municipalities_16 = com_matrix_16.columns.values.tolist()
municipalities_17 = com_matrix_17.columns.values.tolist()
municipalities_18 = com_matrix_18.columns.values.tolist()
municipalities_19 = com_matrix_19.columns.values.tolist()
municipalities_20 = com_matrix_20.columns.values.tolist()

In [13]:
dif_14_15 = list(set(municipalities_14).symmetric_difference(set(municipalities_15)))
print(dif_14_15,'\n', 'The modification are:',len(dif_14_15), '\n',
      'The actual difference should be:',len(com_matrix_14) - len(com_matrix_15))

[265, 282, 365, 458, 491, 568, 608, 612, 623, 643, 644, 1671, 1930, 1931] 
 The modification are: 14 
 The actual difference should be: 10


In [14]:
dif_15_16 = list(set(municipalities_15).symmetric_difference(set(municipalities_16)))
print(dif_15_16,'\n', 'The modification are:', len(dif_15_16), '\n',
      'The actual difference should be:', len(com_matrix_15) - len(com_matrix_16))

[241, 381, 424, 425, 478, 1921, 1940, 1942, 1945] 
 The modification are: 9 
 The actual difference should be: 3


In [15]:
dif_16_17 = list(set(municipalities_16).symmetric_difference(set(municipalities_17)))
print(dif_16_17,'\n', 'The modification are:', len(dif_16_17), '\n',
      'The actual difference should be:', len(com_matrix_16) - len(com_matrix_17))

[844, 846, 860, 1948] 
 The modification are: 4 
 The actual difference should be: 2


In [16]:
dif_17_18 = list(set(municipalities_17).symmetric_difference(set(municipalities_18)))
print(dif_17_18,'\n', 'The modification are:', len(dif_17_18), '\n',
      'The actual difference should be:', len(com_matrix_17) - len(com_matrix_18))

[7, 18, 40, 48, 63, 70, 81, 140, 196, 1908, 1949, 1950, 1952, 1987] 
 The modification are: 14 
 The actual difference should be: 8


In [17]:
dif_18_19 = list(set(municipalities_18).symmetric_difference(set(municipalities_19)))
print(dif_18_19,'\n', 'The modification are:', len(dif_18_19), '\n',
      'The actual difference should be:', len(com_matrix_18) - len(com_matrix_19))

[5, 9, 15, 17, 22, 25, 53, 56, 58, 79, 236, 304, 393, 545, 576, 584, 585, 588, 611, 617, 620, 1651, 1663, 689, 1722, 707, 733, 738, 870, 874, 881, 1927, 1954, 1959, 1960, 1961, 1963, 1966, 1969, 1970, 951, 1978, 962] 
 The modification are: 43 
 The actual difference should be: 25


In [18]:
dif_19_20 = list(set(municipalities_19).symmetric_difference(set(municipalities_20)))
print(dif_19_20,'\n', 'The modification are:', len(dif_19_20), '\n',
      'The actual difference should be:', len(com_matrix_19) - len(com_matrix_20))

[] 
 The modification are: 0 
 The actual difference should be: 0


## Backcasting border changes

Every year, some municipalities disappear. Sometimes they merge with other existent municipalities or sometimes a new bigger municipality is created by the fusion of others. In this section we will write a small funciton that will take a given year commuting matrix, the disappearing municipalities and the new one. Then we will create another function that does the same for the other type of DataFrame, the one that also has the distance between municipalities.

In [19]:
def commuting_matrix_updater(commuting_matrix, disappearing_municipality, new_municipality):
    df = commuting_matrix.copy()
    municipalities = df.columns.values.tolist()
    print
    if new_municipality in municipalities:
        df[new_municipality] += df[disappearing_municipality]
        del df[disappearing_municipality]
        df = df.T
        df[new_municipality] += df[disappearing_municipality]
        del df[disappearing_municipality]
    else:
        df[new_municipality] = df[disappearing_municipality]
        del df[disappearing_municipality]
        df = df.T
        df[new_municipality] = df[disappearing_municipality]
        del df[disappearing_municipality]
    return df

In [20]:
def mobility_dataframe_updater(mobility_dataframe, disappearing_municipalities, new_municipalities):
    return

Here, we already have a function that can modify the commuting matrix in the way we mentioned. To avoid problems with the division of municipalities and merging of many into one, we can write a separate loop.

### 2014 - 2020

2014 - 2015

In [21]:
print(len(com_matrix_14))

403


In [22]:
disappearing_municipalities = [491, 608, 623, 643, 644]
target_municipality = 1931
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [265, 282]
target_municipality = 241
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [365, 458]
target_municipality = 361
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [568, 612]
target_municipality = 1930
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)
    
# Code for the last correction

print(len(com_matrix_14))

394


Here, I still have to fix a very unusual change in borders that happened at the end of the year. The municipality 1671 got disolved and the municipality 796 took 6719 inhabitants and the municipality 828 took 4629 inhabitants. Therefore there must be a discrepancy of 1 extra municipality in the 2014 matrix.

2015 - 2016

In [23]:
disappearing_municipalities = [381, 424, 425]
target_municipality = 1942
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [478]
target_municipality = 385
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [241]
target_municipality = 1945
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [1921]
target_municipality = 1940
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)
print(len(com_matrix_14))

391


2016 - 2017

In [24]:
disappearing_municipalities = [844, 846, 860]
target_municipality = 1948
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)
print(len(com_matrix_14))

389


2017 - 2018

In [25]:
disappearing_municipalities = [7, 48]
target_municipality = 1950
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [1987, 18, 40]
target_municipality = 1952
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [81, 140]
target_municipality = 80
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [196]
target_municipality = 299
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [63, 70, 1908]
target_municipality = 1949
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)
print(len(com_matrix_14))

381


2018 - 2019

In [26]:
disappearing_municipalities = [53, 5, 1651, 1663]
target_municipality = 1966
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [545, 707, 620]
target_municipality = 1961
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [15, 22, 25, 56]
target_municipality = 1969
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [58, 79, 1722]
target_municipality = 1970
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [689, 1927]
target_municipality = 1978
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [236, 304, 733]
target_municipality = 1960
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [17, 9]
target_municipality = 14
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [393]
target_municipality = 394
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [576]
target_municipality = 575
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [584, 585, 588, 611, 617]
target_municipality = 1963
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [738, 870, 874]
target_municipality = 1959
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [962, 881, 951]
target_municipality = 1954
for i in range(len(disappearing_municipalities)):
    com_matrix_14 = commuting_matrix_updater(com_matrix_14, disappearing_municipalities[i], target_municipality)
print(len(com_matrix_14))

356


### 2015 - 2020

2015 - 2016

In [27]:
disappearing_municipalities = [381, 424, 425]
target_municipality = 1942
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [478]
target_municipality = 385
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [241]
target_municipality = 1945
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [1921]
target_municipality = 1940
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)
print(len(com_matrix_15))

390


2016 - 2017

In [28]:
disappearing_municipalities = [844, 846, 860]
target_municipality = 1948
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)
print(len(com_matrix_15))

388


2017 - 2018

In [29]:
disappearing_municipalities = [7, 48]
target_municipality = 1950
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [1987, 18, 40]
target_municipality = 1952
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [81, 140]
target_municipality = 80
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [196]
target_municipality = 299
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [63, 70, 1908]
target_municipality = 1949
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)
print(len(com_matrix_15))

380


2018 - 2019

In [30]:
disappearing_municipalities = [53, 5, 1651, 1663]
target_municipality = 1966
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [545, 707, 620]
target_municipality = 1961
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [15, 22, 25, 56]
target_municipality = 1969
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [58, 79, 1722]
target_municipality = 1970
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [689, 1927]
target_municipality = 1978
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [236, 304, 733]
target_municipality = 1960
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [17, 9]
target_municipality = 14
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [393]
target_municipality = 394
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [576]
target_municipality = 575
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [584, 585, 588, 611, 617]
target_municipality = 1963
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [738, 870, 874]
target_municipality = 1959
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [962, 881, 951]
target_municipality = 1954
for i in range(len(disappearing_municipalities)):
    com_matrix_15 = commuting_matrix_updater(com_matrix_15, disappearing_municipalities[i], target_municipality)
print(len(com_matrix_15))

355


### 2016 - 2020

2016 - 2017

In [31]:
disappearing_municipalities = [844, 846, 860]
target_municipality = 1948
for i in range(len(disappearing_municipalities)):
    com_matrix_16 = commuting_matrix_updater(com_matrix_16, disappearing_municipalities[i], target_municipality)
print(len(com_matrix_16))

388


2017 - 2018

In [32]:
disappearing_municipalities = [7, 48]
target_municipality = 1950
for i in range(len(disappearing_municipalities)):
    com_matrix_16 = commuting_matrix_updater(com_matrix_16, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [1987, 18, 40]
target_municipality = 1952
for i in range(len(disappearing_municipalities)):
    com_matrix_16 = commuting_matrix_updater(com_matrix_16, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [81, 140]
target_municipality = 80
for i in range(len(disappearing_municipalities)):
    com_matrix_16 = commuting_matrix_updater(com_matrix_16, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [196]
target_municipality = 299
for i in range(len(disappearing_municipalities)):
    com_matrix_16 = commuting_matrix_updater(com_matrix_16, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [63, 70, 1908]
target_municipality = 1949
for i in range(len(disappearing_municipalities)):
    com_matrix_16 = commuting_matrix_updater(com_matrix_16, disappearing_municipalities[i], target_municipality)
print(len(com_matrix_16))

380


2018 - 2019

In [33]:
disappearing_municipalities = [53, 5, 1651, 1663]
target_municipality = 1966
for i in range(len(disappearing_municipalities)):
    com_matrix_16 = commuting_matrix_updater(com_matrix_16, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [545, 707, 620]
target_municipality = 1961
for i in range(len(disappearing_municipalities)):
    com_matrix_16 = commuting_matrix_updater(com_matrix_16, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [15, 22, 25, 56]
target_municipality = 1969
for i in range(len(disappearing_municipalities)):
    com_matrix_16 = commuting_matrix_updater(com_matrix_16, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [58, 79, 1722]
target_municipality = 1970
for i in range(len(disappearing_municipalities)):
    com_matrix_16 = commuting_matrix_updater(com_matrix_16, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [689, 1927]
target_municipality = 1978
for i in range(len(disappearing_municipalities)):
    com_matrix_16 = commuting_matrix_updater(com_matrix_16, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [236, 304, 733]
target_municipality = 1960
for i in range(len(disappearing_municipalities)):
    com_matrix_16 = commuting_matrix_updater(com_matrix_16, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [17, 9]
target_municipality = 14
for i in range(len(disappearing_municipalities)):
    com_matrix_16 = commuting_matrix_updater(com_matrix_16, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [393]
target_municipality = 394
for i in range(len(disappearing_municipalities)):
    com_matrix_16 = commuting_matrix_updater(com_matrix_16, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [576]
target_municipality = 575
for i in range(len(disappearing_municipalities)):
    com_matrix_16 = commuting_matrix_updater(com_matrix_16, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [584, 585, 588, 611, 617]
target_municipality = 1963
for i in range(len(disappearing_municipalities)):
    com_matrix_16 = commuting_matrix_updater(com_matrix_16, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [738, 870, 874]
target_municipality = 1959
for i in range(len(disappearing_municipalities)):
    com_matrix_16 = commuting_matrix_updater(com_matrix_16, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [962, 881, 951]
target_municipality = 1954
for i in range(len(disappearing_municipalities)):
    com_matrix_16 = commuting_matrix_updater(com_matrix_16, disappearing_municipalities[i], target_municipality)
print(len(com_matrix_16))

355


### 2017 - 2020

2017 - 2018

In [34]:
disappearing_municipalities = [7, 48]
target_municipality = 1950
for i in range(len(disappearing_municipalities)):
    com_matrix_17 = commuting_matrix_updater(com_matrix_17, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [1987, 18, 40]
target_municipality = 1952
for i in range(len(disappearing_municipalities)):
    com_matrix_17 = commuting_matrix_updater(com_matrix_17, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [81, 140]
target_municipality = 80
for i in range(len(disappearing_municipalities)):
    com_matrix_17 = commuting_matrix_updater(com_matrix_17, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [196]
target_municipality = 299
for i in range(len(disappearing_municipalities)):
    com_matrix_17 = commuting_matrix_updater(com_matrix_17, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [63, 70, 1908]
target_municipality = 1949
for i in range(len(disappearing_municipalities)):
    com_matrix_17 = commuting_matrix_updater(com_matrix_17, disappearing_municipalities[i], target_municipality)
print(len(com_matrix_17))

380


2018 - 2019

In [35]:
disappearing_municipalities = [53, 5, 1651, 1663]
target_municipality = 1966
for i in range(len(disappearing_municipalities)):
    com_matrix_17 = commuting_matrix_updater(com_matrix_17, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [545, 707, 620]
target_municipality = 1961
for i in range(len(disappearing_municipalities)):
    com_matrix_17 = commuting_matrix_updater(com_matrix_17, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [15, 22, 25, 56]
target_municipality = 1969
for i in range(len(disappearing_municipalities)):
    com_matrix_17 = commuting_matrix_updater(com_matrix_17, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [58, 79, 1722]
target_municipality = 1970
for i in range(len(disappearing_municipalities)):
    com_matrix_17 = commuting_matrix_updater(com_matrix_17, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [689, 1927]
target_municipality = 1978
for i in range(len(disappearing_municipalities)):
    com_matrix_17 = commuting_matrix_updater(com_matrix_17, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [236, 304, 733]
target_municipality = 1960
for i in range(len(disappearing_municipalities)):
    com_matrix_17 = commuting_matrix_updater(com_matrix_17, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [17, 9]
target_municipality = 14
for i in range(len(disappearing_municipalities)):
    com_matrix_17 = commuting_matrix_updater(com_matrix_17, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [393]
target_municipality = 394
for i in range(len(disappearing_municipalities)):
    com_matrix_17 = commuting_matrix_updater(com_matrix_17, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [576]
target_municipality = 575
for i in range(len(disappearing_municipalities)):
    com_matrix_17 = commuting_matrix_updater(com_matrix_17, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [584, 585, 588, 611, 617]
target_municipality = 1963
for i in range(len(disappearing_municipalities)):
    com_matrix_17 = commuting_matrix_updater(com_matrix_17, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [738, 870, 874]
target_municipality = 1959
for i in range(len(disappearing_municipalities)):
    com_matrix_17 = commuting_matrix_updater(com_matrix_17, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [962, 881, 951]
target_municipality = 1954
for i in range(len(disappearing_municipalities)):
    com_matrix_17 = commuting_matrix_updater(com_matrix_17, disappearing_municipalities[i], target_municipality)
print(len(com_matrix_17))

355


### 2018 - 2020

2018 - 2019

In [36]:
disappearing_municipalities = [53, 5, 1651, 1663]
target_municipality = 1966
for i in range(len(disappearing_municipalities)):
    com_matrix_18 = commuting_matrix_updater(com_matrix_18, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [545, 707, 620]
target_municipality = 1961
for i in range(len(disappearing_municipalities)):
    com_matrix_18 = commuting_matrix_updater(com_matrix_18, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [15, 22, 25, 56]
target_municipality = 1969
for i in range(len(disappearing_municipalities)):
    com_matrix_18 = commuting_matrix_updater(com_matrix_18, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [58, 79, 1722]
target_municipality = 1970
for i in range(len(disappearing_municipalities)):
    com_matrix_18 = commuting_matrix_updater(com_matrix_18, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [689, 1927]
target_municipality = 1978
for i in range(len(disappearing_municipalities)):
    com_matrix_18 = commuting_matrix_updater(com_matrix_18, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [236, 304, 733]
target_municipality = 1960
for i in range(len(disappearing_municipalities)):
    com_matrix_18 = commuting_matrix_updater(com_matrix_18, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [17, 9]
target_municipality = 14
for i in range(len(disappearing_municipalities)):
    com_matrix_18 = commuting_matrix_updater(com_matrix_18, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [393]
target_municipality = 394
for i in range(len(disappearing_municipalities)):
    com_matrix_18 = commuting_matrix_updater(com_matrix_18, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [576]
target_municipality = 575
for i in range(len(disappearing_municipalities)):
    com_matrix_18 = commuting_matrix_updater(com_matrix_18, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [584, 585, 588, 611, 617]
target_municipality = 1963
for i in range(len(disappearing_municipalities)):
    com_matrix_18 = commuting_matrix_updater(com_matrix_18, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [738, 870, 874]
target_municipality = 1959
for i in range(len(disappearing_municipalities)):
    com_matrix_18 = commuting_matrix_updater(com_matrix_18, disappearing_municipalities[i], target_municipality)

disappearing_municipalities = [962, 881, 951]
target_municipality = 1954
for i in range(len(disappearing_municipalities)):
    com_matrix_18 = commuting_matrix_updater(com_matrix_18, disappearing_municipalities[i], target_municipality)
print(len(com_matrix_18))

355


### 2019 - 2020

There is no changes in municipalities between 2019 and 2020, therefore the DataFrames stay unmodified.

In [37]:
com_matrix_14

,3,10,14,24,34,37,47,50,59,60,...,1949,1966,1961,1969,1970,1978,1960,1963,1959,1954
3,1100.0,500.0,300.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,1100.0,4300.0,900.0,400.0,0.0,100.0,200.0,0.0,0.0,100.0,...,0.0,800.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0
14,900.0,1300.0,69300.0,1300.0,100.0,800.0,1700.0,0.0,800.0,0.0,...,400.0,7600.0,0.0,7700.0,800.0,0.0,0.0,0.0,0.0,0.0
24,100.0,200.0,100.0,700.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,0.0,0.0,100.0,0.0,32900.0,0.0,100.0,800.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,800.0,0.0,0.0,5600.0,200.0,100.0,300.0,0.0
1960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,800.0,0.0,0.0,100.0,7400.0,0.0,100.0,0.0
1963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15700.0,0.0,0.0
1959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,200.0,100.0,0.0,9600.0,0.0
